In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
import cactas as C

2023-01-29 21:05:42.336377: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------


In [3]:
model = C.Helper.load_unet('/raid/psych/CACTAS/unet_full_33_cases_weights.hdf5')

2023-01-29 21:05:43.753336: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-01-29 21:05:43.841195: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:47:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2023-01-29 21:05:43.841240: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-01-29 21:05:43.844468: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2023-01-29 21:05:43.844504: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublasLt.so.11
2023-01-29 21:05:43.845351: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcufft.so.10
2

In [4]:
TEST_DATA = '/raid/mpsych/CACTAS/DATA/Nathan_Test/'
TEST_DATA_OUT = '/raid/mpsych/CACTAS/DATA/Nathan_Test_Predicted2/'
TEST_DATA_IGNORE = ['41','51','61'] # z-spacing different

In [5]:
import os
TEST_SCANS = sorted(os.listdir(TEST_DATA))

In [6]:
THRESHOLD = .99999
# 0.8
# ESUS 10 0.483503737542056 0.493806953687987
# Stored /raid/mpsych/CACTAS/DATA/Nathan_Test_Predicted/10.npy
# ESUS 14 0.34935670820052156 0.47393831164348005
# Stored /raid/mpsych/CACTAS/DATA/Nathan_Test_Predicted/14.npy
# ESUS 15 0.5743982356256112 0.489993468153211
# Stored /raid/mpsych/CACTAS/DATA/Nathan_Test_Predicted/15.npy

# 0.9
# ESUS 10 0.5459574137741867 0.4920005726316911
# Stored /raid/mpsych/CACTAS/DATA/Nathan_Test_Predicted/10.npy
# ESUS 14 0.4118643408780516 0.48950696804557564
# Stored /raid/mpsych/CACTAS/DATA/Nathan_Test_Predicted/14.npy
# ESUS 15 0.6268240963507236 0.4794244698760051
# Stored /raid/mpsych/CACTAS/DATA/Nathan_Test_Predicted/15.npy

# 0.95
# ESUS 10 0.5955386836555591 0.4852241766819561
# Stored /raid/mpsych/CACTAS/DATA/Nathan_Test_Predicted/10.npy
# ESUS 14 0.4546363286847716 0.495436773200697
# Stored /raid/mpsych/CACTAS/DATA/Nathan_Test_Predicted/14.npy
# ESUS 15 0.6791448265509088 0.4630452527458047
# Stored /raid/mpsych/CACTAS/DATA/Nathan_Test_Predicted/15.npy

# 0.99
# ESUS 10 0.6770774116374255 0.46329788505712477
# Stored /raid/mpsych/CACTAS/DATA/Nathan_Test_Predicted/10.npy
# ESUS 14 0.6061867171110868 0.48672684666811006
# Stored /raid/mpsych/CACTAS/DATA/Nathan_Test_Predicted/14.npy
# ESUS 15 0.7479408636903893 0.43192325685362054
# Stored /raid/mpsych/CACTAS/DATA/Nathan_Test_Predicted/15.npy

In [7]:
JACCARDS = {}
TIMINGS = {}

for i,s in enumerate(TEST_SCANS[0:3]):
    
    s_stripped = s.replace('ESUS ','')
    
    if s_stripped in TEST_DATA_IGNORE:
        continue 

    image, label = C.Util.load(i, datapath=TEST_DATA)
    

    label_pred = np.zeros(label.shape, dtype=bool)
    label_pred[:,:,:] = 0
    
    jaccard_per_slice = []
    total_time = 0
    
    for z in range(image.shape[2]):
        
        t0 = time.time()

        cur_slice = image[:,:,z]

        cur_slice = C.Util.normalize(cur_slice)

        cur_slice_pred = model.predict(cur_slice.reshape(1,512,512,1))

        cur_slice_pred = C.Util.binarize(cur_slice_pred[0,:,:,0], threshold=THRESHOLD)

        label_pred[:,:,z] = cur_slice_pred

        total_time += time.time()-t0
        
        jac = C.Util.jaccard(C.Util.binarize(label[:,:,z]), cur_slice_pred)

        jaccard_per_slice.append(jac)
        
    TIMINGS[s] = total_time
    JACCARDS[s] = jaccard_per_slice
    print(s, np.mean(jaccard_per_slice), np.std(jaccard_per_slice))
    
    np.save(TEST_DATA_OUT+s_stripped+'.npy', label_pred)
    print('Stored', TEST_DATA_OUT+s_stripped+'.npy')

2023-01-29 21:05:54.548740: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-01-29 21:05:54.569167: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2245645000 Hz
2023-01-29 21:05:54.743309: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudnn.so.8
2023-01-29 21:05:55.354886: I tensorflow/stream_executor/cuda/cuda_dnn.cc:359] Loaded cuDNN version 8201
2023-01-29 21:05:55.997763: W tensorflow/stream_executor/gpu/asm_compiler.cc:191] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.0
2023-01-29 21:05:55.997788: W tensorflow/stream_executor/gpu/asm_compiler.cc:194] Used ptxas at ptxas
2023-01-29 21:05:55.997852: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Unimplemented: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modif

ESUS 10 0.8675213675213675 0.3390103898337393
Stored /raid/mpsych/CACTAS/DATA/Nathan_Test_Predicted2/10.npy
ESUS 14 0.9373440285204991 0.2421633741121393
Stored /raid/mpsych/CACTAS/DATA/Nathan_Test_Predicted2/14.npy
ESUS 15 0.9171597633136095 0.2756405845882158
Stored /raid/mpsych/CACTAS/DATA/Nathan_Test_Predicted2/15.npy


In [10]:
TIMINGS

{'ESUS 10': 12.128159284591675,
 'ESUS 14': 13.243142366409302,
 'ESUS 15': 8.6052885055542,
 'ESUS 2': 12.593088626861572,
 'ESUS 27': 11.073944091796875,
 'ESUS 32': 12.300761222839355,
 'ESUS 33': 13.360402584075928,
 'ESUS 35': 12.29096007347107,
 'ESUS 39': 12.941060066223145,
 'ESUS 46': 11.964716911315918,
 'ESUS 47': 9.917606592178345,
 'ESUS 48': 27.16199278831482,
 'ESUS 5': 10.339241743087769,
 'ESUS 50': 12.533114671707153,
 'ESUS 52': 11.832925081253052,
 'ESUS 53': 12.148744344711304,
 'ESUS 57': 12.239945411682129,
 'ESUS 64': 12.09887957572937,
 'ESUS 66': 12.309496641159058,
 'ESUS 73': 13.097173929214478,
 'ESUS 75': 12.072381973266602,
 'ESUS 83': 13.584279298782349,
 'ESUS 84': 8.751081466674805,
 'ESUS 86': 12.256171941757202,
 'ESUS 9': 12.360882997512817,
 'ESUS 91': 8.96485447883606,
 'ESUS 93': 12.672183513641357,
 'ESUS 94': 13.133982419967651}

In [11]:
import pickle

In [12]:
with open('/raid/mpsych/CACTAS/unet_full_099_results.pkl','wb') as f:
    pickle.dump([JACCARDS, TIMINGS], f)